In [72]:
%load_ext autoreload
%autoreload 2

import sys

PROJECT_PATH = "/notebooks/evaluating-cmr-in-mm/"
sys.path.append(PROJECT_PATH)
import torch
import os
from munch import Munch
import pandas as pd
import argparse

torch.set_num_threads(4)
from src.data.dataset import Dataset
from src.utils.dataset_preprocessing import load_json_annotations
from src.models.encoders.clip import CLIP
from src.retrieval.retriever import Retriever
from src.metrics.recall_at_k import recall_at_k
from src.models.relevance_estimators.clip_based import RelevanceEstimator
from src.metrics.dcg import DCG
from src.utils.dataset_preprocessing import save_results_dataframe

os.environ["http_proxy"] = "http://devproxy.bloomberg.com:82"
os.environ["https_proxy"] = "http://devproxy.bloomberg.com:82"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import inspect 
from src.models.encoders.clip import CLIP

print(inspect.getsource(CLIP.compute_caption_embeddings))

    def compute_caption_embeddings(self, ds_split: Type[Dataset]) -> (List[int], List[str], torch.Tensor):
        """Compute caption embeddings for a given dataset

        Args:
            self (src.models.encoders.clip.CLIP): instance of the class
            ds_split (Type[Dataset]): dataset split for extracting caption ids and raw captions

        Returns:
            List[int]: list of caption ids
            List[int]: list of raw captions
            torch.Tensor: tensor of shape (n, m) where n - number of captions, m - caption embedding size
        """
        # check if the path already exists
        emb_path = get_precomputed_embeddings_path(config=self.config, dtype='capt')

        if os.path.exists(emb_path):
            print('Caption embeddings are already precomputed')
            caption_ids, capts, capt_emb = load_filenames_embs_from_pkl(emb_file_path=emb_path)
            assert len(caption_ids) == len(capts) == capt_emb.shape[0]
            return caption_ids, 

In [31]:
# load precomputed image embeddings
data = model.compute_caption_embeddings(
    ds_split=ds_test_split
)

Computing caption embeddings...
Test


Batches: 100%|██████████| 40/40 [00:00<00:00, 43.94it/s]

125


In [33]:
len(data)

3

In [4]:
PROJECT_PATH = "/notebooks/evaluating-cmr-in-mm/"
sys.path.append(PROJECT_PATH)

In [14]:
dataset_type = 'f30k'

config_filename = f"development_{dataset_type}_dsp.yaml"
CONFIG_PATH = os.path.join("../config/clip", config_filename)
print("Config path: ", CONFIG_PATH)

with open(CONFIG_PATH, "rb") as f:
    config = Munch.fromYAML(f)

print(
    "Loading the annotations, preparing the dataset, the model, image embeddings..."
)
json_file = load_json_annotations(config=config)

ds_test_split = Dataset(config=config, split="test", json_file=json_file)

# load the model
model = CLIP(config=config)
# load precomputed image embeddings
data = model.compute_caption_embeddings(
    ds_split=ds_test_split
)

rel_estimator = RelevanceEstimator(config=config, dataset=ds_test_split)
retriever = Retriever(config=config, model=model)
dcg = DCG(config=config, rel_estimator=rel_estimator)

i2t_queries = []
i2t_targets = []
i2t_retrieved_documents = []
i2t_scores = []
i2t_recalls_at_1 = []
i2t_recalls_at_5 = []
i2t_recalls_at_10 = []
i2t_dcgs = []

Config path:  ../config/clip/development_f30k_dsp.yaml
Loading the annotations, preparing the dataset, the model, image embeddings...
Loaded annotations from  /notebooks/evaluating-cmr-in-mm/f30k/annotations/dataset_flickr30k.json
Computing caption embeddings...


Batches: 100%|██████████| 40/40 [00:00<00:00, 43.55it/s]


In [36]:
from src.utils.dataset_preprocessing import get_emb_file_path, dump_filenames_embs_to_pkl, load_filenames_embs_from_pkl

emb_path = get_emb_file_path(config, 'capt')

dump_filenames_embs_to_pkl(emb_path, data)

Saved files to  /notebooks/evaluating-cmr-in-mm/f30k/f30k-capt-embeddings.pkl


In [37]:
test = load_filenames_embs_from_pkl(emb_path)



Loaded precomputed filenames and embeddings from  /notebooks/evaluating-cmr-in-mm/f30k/f30k-capt-embeddings.pkl


In [39]:
import torch

def get_device():
    """

    :return:
    """
    return 'cuda' if torch.cuda.is_available() else 'cpu' if torch.backends.mps.is_built() else 'cpu'

get_device()

'cuda'

In [62]:
def define_machine_type(device) -> str:
    if device == 'cuda':
        return 'dsp'
    elif device == 'cpu':
        return 'local'
    else:
        raise NotImplementedError

In [61]:
config.dataset

Munch({'root': '/notebooks/evaluating-cmr-in-mm/f30k/', 'img_folder': 'flickr30k-images', 'img_emb_filename': 'f30k-img-embeddings.pkl', 'capt_emb_filename': 'f30k-capt-embeddings.pkl', 'annotation_file': 'annotations/dataset_flickr30k.json'})

In [65]:
from src.utils.utils import get_device, get_machine_type
import os

dataset='f30k'
model='clip'

def get_config_path(dataset, model):
    machine_type = get_machine_type()
    config_filename = f"development_{dataset}_{machine_type}.yaml"
    path = os.path.join(f"./config/{model}", config_filename)
    return path

get_config_path(dataset, model)

'./config/clip/development_f30k_dsp.yaml'

In [68]:
os.path.exists("./config/clip/development_f30k_dsp.yaml")

False

In [70]:
PROJECT_PATH = "../"
sys.path.append(PROJECT_PATH)

In [71]:
! pwd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/notebooks/evaluating-cmr-in-mm/notebooks


In [18]:
# load the model
model = CLIP(config=config)
# load precomputed image embeddings
data = model.compute_caption_embeddings(
    ds_split=ds_test_split
)

Computing caption embeddings...


Batches: 100%|██████████| 40/40 [00:00<00:00, 44.27it/s]


In [19]:
data